In [2]:
import pandas as pd
import numpy as np

In [3]:
scores=pd.read_excel('/rds/general/user/hsl121/home/hda_project/data/Scores 6 Jan 2025_Prescribed_Completed Baseline PROMs.xlsx')
order_states=pd.read_csv('/rds/general/user/hsl121/home/hda_project/data/2025 AI Imperial Dataset Orders_Prescription_Formulas.csv')
prescription_formula=pd.read_csv('/rds/general/user/hsl121/home/hda_project/data/2025 AI Imperial Dataset Orders_Prescription_Formulas.csv')
adverse=pd.read_excel('/rds/general/user/hsl121/home/hda_project/data/Adverse Events 6 Jan 2025_Prescribed_Completed Baseline PROMs.xlsx')
comorbidites=pd.read_excel('/rds/general/user/hsl121/home/hda_project/data/Comorbidities 6 Jan 2025_Prescribed_Completed Baseline PROMs.xlsx')
medications=pd.read_excel('/rds/general/user/hsl121/home/hda_project/data/Medications 6 Jan 2025_Prescribed_Completed Baseline PROMs.xlsx')
prescription=pd.read_excel('/rds/general/user/hsl121/home/hda_project/data/Prescriptions 6 Jan 2025_Prescribed_Completed Baseline PROMs NEW.xlsx')
product=pd.read_excel('/rds/general/user/hsl121/home/hda_project/data/Product Data.xlsx')

EQ5D Score Mapping

In [4]:
eq5d=scores[scores['promName']=='EQ5D']
eq5d=eq5d[['SID','Round','promName','total_score','completionDate']]

In [5]:
eq5d["SID"].nunique()

31434

In [6]:
print(eq5d.columns)
# keeping SID with follow up data
sids_with_followup = eq5d[eq5d['Round'].astype(int) > 1]['SID'].unique()
eq5d = eq5d[eq5d['SID'].isin(sids_with_followup)].reset_index(drop=True)



Index(['SID', 'Round', 'promName', 'total_score', 'completionDate'], dtype='object')


In [6]:
xls = pd.ExcelFile('/rds/general/user/hsl121/home/hda_project/Protocol and data framework/Copy of EQ-5D-5L_Crosswalk_Index_Value_Calculator_MAC.xls')
print(xls.sheet_names)


['Title', 'Index Value Calculator', 'EQ-5D-5L Value Sets']


In [7]:
eq5d_index = pd.read_excel(xls, sheet_name='EQ-5D-5L Value Sets', skiprows=1)


In [8]:
eq5d_index = eq5d_index.drop(eq5d_index.columns[0], axis=1)
eq5d_index= eq5d_index[['5L profile', 'UK']]


In [9]:
eq5d_index.columns

Index(['5L profile', 'UK'], dtype='object')

In [10]:
eq5d.columns

Index(['SID', 'Round', 'promName', 'total_score', 'completionDate'], dtype='object')

In [11]:
eq5d = eq5d.merge(eq5d_index, left_on='total_score', right_on='5L profile', how='left')


In [ ]:
eq5d.head

<bound method NDFrame.head of                                      SID  Round promName  total_score  \
0           003adIEhRDbcWiL9YN4rjAvekGl1      1     EQ5D      43455.0   
1           003adIEhRDbcWiL9YN4rjAvekGl1      2     EQ5D      31222.0   
2           003adIEhRDbcWiL9YN4rjAvekGl1      3     EQ5D      33222.0   
3       00b622bb2e064de91714bb4d7812245e      1     EQ5D      14424.0   
4       00b622bb2e064de91714bb4d7812245e      2     EQ5D      24434.0   
...                                  ...    ...      ...          ...   
107521              zZwQ0lYo31weDN038pOq      1     EQ5D      11333.0   
107522              zZwQ0lYo31weDN038pOq      2     EQ5D      11222.0   
107523              zZwQ0lYo31weDN038pOq      3     EQ5D      11122.0   
107524      zZXvDhEI5neNrU419i5tlcvlEkz1      1     EQ5D      32544.0   
107525      zZXvDhEI5neNrU419i5tlcvlEkz1      2     EQ5D      21222.0   

                  completionDate  5L profile     UK  
0       2023-01-27T09:28:22.807Z     43

: 

In [12]:

eq5d_wide = eq5d.pivot_table(index='SID', 
                                    columns='Round', 
                                    values='UK', 
                                    aggfunc='first')

eq5d_wide.columns = [f"EQ5D_Round{r}" for r in eq5d_wide.columns]
eq5d_wide = eq5d_wide.reset_index()


In [ ]:
eq5d_wide.columns
eq5d_wide.shape

(25362, 14)

: 

In [13]:
# Pivot 1: UK EQ5D scores
eq5d_scores = eq5d.pivot_table(
    index='SID',
    columns='Round',
    values='UK',
    aggfunc='first'
)
eq5d_scores.columns = [f"EQ5D_Round{r}" for r in eq5d_scores.columns]
eq5d_scores = eq5d_scores.reset_index()

# Pivot 2: Completion dates
eq5d_dates = eq5d.pivot_table(
    index='SID',
    columns='Round',
    values='completionDate',
    aggfunc='first'
)
eq5d_dates.columns = [f"EQ5D_Date_Round{r}" for r in eq5d_dates.columns]
eq5d_dates = eq5d_dates.reset_index()

# Merge both tables
eq5d_wide_with_dates = pd.merge(eq5d_scores, eq5d_dates, on='SID', how='left')


In [14]:
eq5d_wide_with_dates.shape

(25362, 27)

In [15]:
eq5d_wide_with_dates.columns

Index(['SID', 'EQ5D_Round1', 'EQ5D_Round2', 'EQ5D_Round3', 'EQ5D_Round4',
       'EQ5D_Round5', 'EQ5D_Round6', 'EQ5D_Round7', 'EQ5D_Round8',
       'EQ5D_Round9', 'EQ5D_Round10', 'EQ5D_Round11', 'EQ5D_Round12',
       'EQ5D_Round13', 'EQ5D_Date_Round1', 'EQ5D_Date_Round2',
       'EQ5D_Date_Round3', 'EQ5D_Date_Round4', 'EQ5D_Date_Round5',
       'EQ5D_Date_Round6', 'EQ5D_Date_Round7', 'EQ5D_Date_Round8',
       'EQ5D_Date_Round9', 'EQ5D_Date_Round10', 'EQ5D_Date_Round11',
       'EQ5D_Date_Round12', 'EQ5D_Date_Round13'],
      dtype='object')

Prescription Data

In [16]:
# Step 1: Prep
eq5d = eq5d_wide_with_dates.copy()
prescription['DateofPrescription'] = pd.to_datetime(prescription['DateofPrescription'])

# Make sure EQ5D dates are datetime and timezone-naive
date_cols = [col for col in eq5d.columns if col.startswith('EQ5D_Date_Round')]
eq5d[date_cols] = eq5d[date_cols].apply(pd.to_datetime, errors='coerce').apply(lambda x: x.dt.tz_localize(None))

# Sort prescriptions
prescription_sorted = prescription.sort_values(['SID', 'DateofPrescription'])
prescription_sorted['prescription_rank'] = prescription_sorted.groupby('SID').cumcount() + 1

# Initialize effective prescription with NaT
eq5d['EffectivePrescriptionDate'] = pd.NaT

# Loop through prescription attempts (1st to 5th)
for attempt in range(1, 10):
    print(f"Checking prescription attempt #{attempt}...")
    
    # Get nth prescription
    nth_presc = prescription_sorted[prescription_sorted['prescription_rank'] == attempt]
    nth_presc = nth_presc[['SID', 'DateofPrescription']].rename(columns={'DateofPrescription': f'PrescriptionDate_{attempt}'})
    
    # Merge into eq5d
    eq5d = eq5d.merge(nth_presc, on='SID', how='left')
    
    # Fill EffectivePrescriptionDate where it's still missing
    mask = eq5d['EffectivePrescriptionDate'].isna() & eq5d[f'PrescriptionDate_{attempt}'].notna()
    eq5d.loc[mask, 'EffectivePrescriptionDate'] = eq5d.loc[mask, f'PrescriptionDate_{attempt}']
    
    # Recalculate timing validity
    eq5d['has_valid_baseline'] = eq5d['EQ5D_Date_Round1'] < eq5d['EffectivePrescriptionDate']
    eq5d['has_valid_followup'] = eq5d[date_cols[1:]].gt(eq5d['EffectivePrescriptionDate'], axis=0).any(axis=1)
    
    # Stop if everyone is now valid
    if eq5d['has_valid_baseline'].all() & eq5d['has_valid_followup'].all():
        print(f"All patients valid by attempt #{attempt}.")
        break


Checking prescription attempt #1...
Checking prescription attempt #2...
Checking prescription attempt #3...
Checking prescription attempt #4...
Checking prescription attempt #5...
Checking prescription attempt #6...
Checking prescription attempt #7...
Checking prescription attempt #8...
Checking prescription attempt #9...


In [17]:
# Step 1: Filter valid patients
valid_eq5d = eq5d[eq5d['has_valid_baseline'] & eq5d['has_valid_followup']].copy()

# Step 2: Get all prescriptions for valid SIDs
valid_sids = valid_eq5d['SID'].unique()
prescription_valid = prescription[prescription['SID'].isin(valid_sids)].copy()

# Step 3: Sort and rank prescriptions per SID
prescription_valid = prescription_valid.sort_values(['SID', 'DateofPrescription'])
prescription_valid['prescription_rank'] = prescription_valid.groupby('SID').cumcount() + 1
prescription_valid = prescription_valid[prescription_valid['prescription_rank'] <= 3]


# Step 4: Pivot each prescription field into separate columns
pivoted_prescription = prescription_valid.pivot(index='SID', columns='prescription_rank', 
                                                 values=['DateofPrescription', 'Product', 'Totalquantity'])

# Step 5: Flatten MultiIndex column names
pivoted_prescription.columns = [f"{var}_Prescription{rank}" for var, rank in pivoted_prescription.columns]
pivoted_prescription = pivoted_prescription.reset_index()

# Step 6: Merge into final wide dataset
final_df = valid_eq5d.merge(pivoted_prescription, on='SID', how='left')


In [18]:
full=pd.merge(final_df, comorbidites, on='SID', how='left')

Other Scores (Baseline) Data

In [19]:
print(scores['promName'].unique())
# PROMs to extract
target_proms = ['GAD7', 'insomniaEfficacyMeasure']

## PGIC was not used because it is not availble as baseline data, only follow up

# Filter scores data to only these PROMs and Round 1
filtered = scores[(scores['promName'].isin(target_proms)) & (scores['Round'] == 1)]

# Pivot to wide format: one column per PROM, showing total_score at Round 1
prom_wide = filtered.pivot_table(index='SID',
                                 columns='promName',
                                 values='total_score',
                                 aggfunc='first')

# Rename columns to include "Round1"
prom_wide.columns = [f"{col}_Round1" for col in prom_wide.columns]

# Reset index to turn SID back into a column
prom_wide = prom_wide.reset_index()


['EQ5D' 'GAD7' 'MPQ2' 'VASPainScore' 'insomniaEfficacyMeasure'
 'painInventoryShortForm' 'PGIC' 'PHQ9' 'MSQoL' 'PDQ39'
 'fibromyalgiaSymptomSeverityScore' 'impactOfEventsScale' 'HIT' 'MIDAS'
 'AKPS' 'POS' 'IBDQ' 'QOLIEQ' 'symptomDistressScale'
 'obsessiveCompulsiveInventory' 'IEPS' 'MOVES' 'SPIN' 'EDEQ']


In [20]:
prom_wide.columns

Index(['SID', 'GAD7_Round1', 'insomniaEfficacyMeasure_Round1'], dtype='object')

In [21]:
full=pd.merge(full, prom_wide, on='SID', how='left')

Final Dataset

In [22]:
date_cols = [col for col in full.columns if col.startswith('EQ5D_Date_Round')]
full = full.drop(columns=date_cols)

In [23]:
drop=['EQ5D_Round7', 'EQ5D_Round8',
       'EQ5D_Round9', 'EQ5D_Round10', 'EQ5D_Round11', 'EQ5D_Round12',
       'EQ5D_Round13','PrescriptionDate_1',
       'has_valid_baseline', 'has_valid_followup', 'PrescriptionDate_2',
       'PrescriptionDate_3', 'PrescriptionDate_4', 'PrescriptionDate_5',
       'PrescriptionDate_6', 'PrescriptionDate_7', 'PrescriptionDate_8',
       'PrescriptionDate_9','Prescribed', 'BaselinePROMS',
       'FirstPrescriptionDate', 'FirstYearPrescribed', 'Status',
       'GovernmentOfficeRegion','DateofPrescription_Prescription1', 'DateofPrescription_Prescription2',
       'DateofPrescription_Prescription3']


full = full.drop(columns=drop)



In [ ]:
full.columns

Index(['SID', 'EQ5D_Round1', 'EQ5D_Round2', 'EQ5D_Round3', 'EQ5D_Round4',
       'EQ5D_Round5', 'EQ5D_Round6', 'EffectivePrescriptionDate',
       'Product_Prescription1', 'Product_Prescription2',
       'Product_Prescription3', 'Totalquantity_Prescription1',
       'Totalquantity_Prescription2', 'Totalquantity_Prescription3', 'Age',
       'Sex', 'occupation', 'weight', 'height', 'BMI', 'primDiag', 'secDiag',
       'tertDiag', 'Myocardial_infarction', 'Congestive_heart_failure',
       'Peripheral_vascular_disease',
       'Cerebrovascular_accident_or_transient_ischemic_attack', 'Dementia',
       'Chronic_obstructive_pulmonary_disease', 'Connective_tissue_disease',
       'Peptic_Ulcer_Disease', 'Liver_disease', 'Diabetes', 'Hemiplegia',
       'Moderate_to_severe_chronic_kidney_disease', 'Solid_tumour', 'Leukemia',
       'Lymphoma', 'AIDS', 'Charlson_comorbidity', 'Hypertension',
       'Depression_or_anxiety', 'Arthritis', 'Epilepsy', 'VTE',
       'Endocrine_thyroid_dysfunction'

: 

In [21]:
# Dataset for EQ5D Score prediction (no adverse events)
full.to_csv("/rds/general/user/hsl121/home/hda_project/rq1/rq1_full_df.csv", index=False) 

NameError: name 'full' is not defined

Adding GAD7 Follow Up Score

In [4]:

gad7=scores[scores['promName']=='GAD7']
gad7=gad7[['SID','Round','promName','total_score','completionDate']]


gad7_wide = gad7.pivot_table(index='SID', 
                                    columns='Round', 
                                    values='total_score', 
                                    aggfunc='first')

gad7_wide.columns = [f"GAD7_Round{r}" for r in gad7_wide.columns]
gad7_wide = gad7_wide.reset_index()

In [ ]:
gad7_wide.head

<bound method NDFrame.head of                                     SID  GAD7_Round1  GAD7_Round2  \
0          003adIEhRDbcWiL9YN4rjAvekGl1         21.0          3.0   
1          00DlmNQ0vYUyGdMbnZu0dbGOeKr2         19.0          NaN   
2                  00UDRrMX0tM2sJ9iQUYR          6.0          4.0   
3          00Z3p6vBtNXuin5DgOCEbqEDLrG2         19.0          NaN   
4      00b622bb2e064de91714bb4d7812245e         17.0         15.0   
...                                 ...          ...          ...   
31467      zzCA5ttM0nYzOR0aDW7un2a1BF42         15.0         13.0   
31468      zzJg4IPT3ZbUPhZweWQCIuIIyDk2         17.0          1.0   
31469      zzVdFjr0w2hziHh0UWLl6x2Hz5Y2         13.0         11.0   
31470      zzZ75pcuChS4zFv3wX4TchiEcVt2         20.0          NaN   
31471              zzoqgNeyWVKrFkwtpebH          7.0          NaN   

       GAD7_Round3  GAD7_Round4  GAD7_Round5  GAD7_Round6  GAD7_Round7  \
0              4.0          NaN          NaN          NaN          

: 